<a href="https://colab.research.google.com/github/dataMethodMan/initialRepo/blob/master/trax_wt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trax
import trax.layers as tl
import trax.data as data 
import trax.supervised as ts 
import trax as trax

     |████████████████████████████████| 629 kB 5.2 MB/s 
     |████████████████████████████████| 4.3 MB 48.0 MB/s 
     |████████████████████████████████| 152 kB 62.8 MB/s 
     |████████████████████████████████| 1.2 MB 48.8 MB/s 
     |████████████████████████████████| 2.5 MB 41.9 MB/s 
     |████████████████████████████████| 249 kB 67.3 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 4.0 MB 42.3 MB/s 
     |████████████████████████████████| 366 kB 55.9 MB/s 
     |████████████████████████████████| 895 kB 65.7 MB/s 
     |████████████████████████████████| 3.3 MB 63.8 MB/s 


In [2]:
import os
import requests

In [3]:
#https://stackoverflow.com/questions/53142474/google-colab-keyerror-colab-tpu-addr
if "TPU_DRIVE_MODE" not in globals():
  url= 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
  resp = requests.post(url)
  TPU_DRIVE_MODE = 1

In [4]:
trax.fastmath.set_backend("jax")

In [5]:
from jax.config import config


In [6]:
config.FLAGS.jax_xla_backend = "tpu_driver"

In [7]:
config.FLAGS.jax_backend_target = "grpc://" + os.environ["COLAB_TPU_ADDR"]

In [8]:
# take some data from imdb and have a little look-see 
imdb =  data.TFDS("imdb_reviews", keys=("text", "label"), train=True)

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:374: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "
/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:387: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "


In [9]:
text_in_bytes, sent = next(imdb())

In [10]:
# convert to utf8 format 
decoded = text_in_bytes.decode("utf-8")
# additinally format the text 
max_len = 100 if len(decoded) > 100 else len(decode)-1
print(decoded[:max_len], sent)

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside.  0


In [16]:
# pipeline = data.Serial(
#     function1("usually loads data"), 
#     function2("does the preprocessing")
# )
# my_data_generator = dataset_generator # function of a generator 
# preprocessing_pipeline = data.Serial(
#     remove_punct_function(puncts_to_remove=".,:;"),
#     stopword_removal_function(stopword_dict="/data/stopword_dict.json"), 
#     my_tokenizer()
# )
# pipeline_generator=preprocessing_pipeline(my_data_generator)
pipeline = data.Serial(
    data.TFDS("imdb_reviews", keys=("text", "label"),train=True), 
    data.Tokenize(vocab_file="en_8k.subword", keys=[0]), 
    data.Shuffle(),
    data.FilterByLength(max_length=128, length_keys=[0]), 
    data.BucketByLength(boundaries= [32,64,128], 
                        batch_sizes = [128, 64, 32], 
                        length_keys=[0]),
    data.AddLossWeights()  #used for masking and padding 

    
)

/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:374: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "
/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:387: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "


In [24]:
# testing our pipeline 
batch_token_ids, batch_target_values, batch_loss_weights = next(pipeline())
# detokensiing the sentences 
first_tokenised_sentence = batch_token_ids[0][:10]
batch_loss_weights = batch_loss_weights[0]
detokenised = data.detokenize(first_tokenised_sentence, vocab_file="en_8k.subword")
# print out our results 
print("""example sentence first 10 tokens: {} ... 
\nExample first 10 tokens after tokenisation: {}... 
\nSentiment: {}
\n batch loss weights: {}""".format(first_tokenised_sentence, 
                          detokenised, 
                          "positive" if batch_target_values[0] else "negative", 
                          batch_loss_weights))

example sentence first 10 tokens: [ 433 1268   96 6300    3  151    4 2156   37 1005] ... 

Example first 10 tokens after tokenisation: As others have mentioned, all the women that go... 

Sentiment: positive

 batch loss weights: 1.0


In [28]:
# download vocab to be used as target example 
import os.path
if not os.path.isfile('/content/pt.wiki.bpe.vs10000.vocab'):
  !wget https://nlp.h-its.org/bpemb/pt/pt.wiki.bpe.vs1000.vocab


# training a s2s model to make a nmt for english to portugese
# using a pretrained model 
seq2seq_model = trax.models.LSTMSeq2SeqAttn(input_vocab_size=data.vocab_size(vocab_file="en_8k.subword"), 
                                           target_vocab_size=data.vocab_size(vocab_type="subword", vocab_file="pt.wiki.bpe.vs1000.vocab", vocab_dir="/content"))
print(seq2seq_model)

--2021-07-20 06:58:30--  https://nlp.h-its.org/bpemb/pt/pt.wiki.bpe.vs1000.vocab
Resolving nlp.h-its.org (nlp.h-its.org)... 193.197.73.80
Connecting to nlp.h-its.org (nlp.h-its.org)|193.197.73.80|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://bpemb.h-its.org/pt/pt.wiki.bpe.vs1000.vocab [following]
--2021-07-20 06:58:31--  https://bpemb.h-its.org/pt/pt.wiki.bpe.vs1000.vocab
Resolving bpemb.h-its.org (bpemb.h-its.org)... 193.197.73.111
Connecting to bpemb.h-its.org (bpemb.h-its.org)|193.197.73.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10378 (10K) [text/plain]
Saving to: ‘pt.wiki.bpe.vs1000.vocab.3’

pt.wiki.bpe.vs1000. 100%[===================>]  10.13K  --.-KB/s    in 0s      

2021-07-20 06:58:32 (187 MB/s) - ‘pt.wiki.bpe.vs1000.vocab.3’ saved [10378/10378]

Serial_in2_out2[
  Select[0,1,0,1]_in2_out4
  Parallel_in2_out2[
    Serial[
      Embedding_8183_512
      LSTM_512
      LSTM_512
    ]
    S

In [31]:
# there are 3 combinerr layers 
# serial (sequential model), branch(parallel model), residual(combination of the two)
# sample implementation 
sentiment_analysis_model = tl.Serial(
    tl.Embedding(data.vocab_size(vocab_file="en_8k.subword"), d_feature=256),
    tl.Mean(axis=1),
    tl.Dense(2),
    tl.Softmax()
)
print(sentiment_analysis_model)


Serial[
  Embedding_8183_256
  Mean
  Dense_2
  Softmax
]
